## Reading the data

Our dataset can be found [here](https://data.world/maxstrange/diplomacyboardgame)

In [1]:
import pandas as pd
import time

In [2]:
# read the dataframes
all_games = pd.read_pickle("data/games.pkl")
all_orders = pd.read_pickle("data/orders.pkl")
all_players = pd.read_pickle("data/players.pkl")
all_turns = pd.read_pickle("data/turns.pkl")
all_units = pd.read_pickle("data/units.pkl")

In [3]:
all_games.head(3)

,id,num_turns,num_players
0,37317,166,7
1,37604,51,7
2,39337,101,7


## Detecting betrayal

What I want to do here is **to detect betrayals within a game**, using the same definitions as in the paper we have studied. Let's recall a few things, and we will explore the dataset based on those definitions.

### What are game actions ? 

Each player has **units** (one per each city a player controls) and thoses are moved using **orders**. There are 2 kinds of orders: 
- **support** order: two units join and become bigger (i.e. stronger). One player can support another.
- movement: move a unit somewhere. If it meets another player's unit, it will be a **battle**

### How to define relationships ? 

Let's follow the definitions given by the paper.

**Act of friendship**: when a player supports another.

**Act of hostility**: When a player invades another, or if a player supports an invasion to the other player's territory.

**Friendship**: a relationship between two players spanning over 3 seasons containing at least 2 **consecutive and reciprocated** acts of friendships.

**Betrayal** / **Broken friendship**: When, after being in a friendship, two players engage in at least 2 acts of hostility. 

### Additional information required for data-processing

It is important to understand the [rules of the game](https://www.playdiplomacy.com/help.php?sub_page=Game_Rules).

Here is a list of points we want to raise before starting the programming, obtained from looking at the rules.
- Each **year** is breaked down into 2 **seasons**: {'Spring', 'Fall'}.
- Each **seasons** is itself divided into several phases, called **turns** (therefore, a year is made of at least 2 turns, and not more than 5)
    - **orders**: each player submit orders to all of its units (that can be **hold**, **move**, **support** or **convoy**)
    - **retreats**: a phase that happens when some units (called **disloged units**) need to retreat. If they can't, they are destroyed
    - **builds**: only happens after the *fall retreat*. Players gain control of SCS they are occupying.
- Geographically, the game is divided into **provinces**
- some provinces are called **supply centers** (SCS) and to win a **player** must control 18 supply centers.
- Each **unit** belongs to a **player** and there can be **only 1 unit** in a province at a time, however **units** can join their force with **support order**.
- There are 2 types of **units**:  {'F' or 'A'} for {Fleet, Army}
- Each **player** is characterized by its country, encoded by a letter: {E,F,I,G,A,T,R} standing for {England, France, Italy, Germany, Austria, Turkey, Russia}

We also give an clarification for the rows of 'all_orders' (i.e. the proper orders) because we will be using those quite a lot, and it can be hard to understand. 
- orders are defined by a **game_id**, a **unit_id** and **turn_number** (which makes sense, considering all the above points). 
- each order has a field **location** which is the province of origin of the unit
- depending on the **unit_order**, here is the description of the fields

| unit_order | location                 | target                            | target_dest     |
| ---------- | ------------------------ | --------------------------------- | --------------- |
| MOVE       | initial loc. of the unit | loc. to move to                   | null            |
| HOLD       | initial loc. of the unit | null                              | null            |
| CONVOY     |                          | initial loc.                      | end goal loc.   |
| SUPPORT    |                          | loc. of unit to be supported      | its target loc. |
| BUILD      | ""                       | encoded string like 'army Berlin' |                 |
| RETREAT    | initial loc. of the unit | target loc                        |                 |
| DESTROY    | initial loc. of the unit |                                   |                 |

### What the map looks like ! 

<img src="img/map.png" width="900">

## Hands on one game

Now that all of this is well-defined, let's see what we can achieve in the code. As it can be quite hard to see how to do this, let's break this down and look at one game.

In [4]:
# extract one game
game = all_games.head(1)
game_id = game.iloc[0,0]
game

,id,num_turns,num_players
0,37317,166,7


In [5]:
# for this game, extract turns, orders and units
turns = all_turns.query("game_id == {}".format(game_id))
orders = all_orders.query("game_id == {}".format(game_id))
units = all_units.query("game_id == {}".format(game_id))
orders

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num
11451415,37317,0,MOVE,Edinburgh,North Sea,None,1,None,1
11451416,37317,1,MOVE,Liverpool,Belgium,None,0,Illegal order replaced with Hold order,1
11451417,37317,2,HOLD,London,None,None,1,None,1
11451418,37317,3,MOVE,Marseilles,Spain,None,1,None,1
11451419,37317,4,MOVE,Paris,Burgundy,None,0,Bounced,1
...,...,...,...,...,...,...,...,...,...
11452959,37317,1,RETREAT,Liverpool,London,None,0,Cant retreat to this location,34
11452964,37317,4,RETREAT,Marseilles,Burgundy,None,1,None,34
11452972,37317,42,RETREAT,Smyrna,Syria,None,1,None,34
11452976,37317,47,BUILD,,army Paris,None,1,None,35


### Finding **acts of friendships**

It's firstly defined by a support. However it is not enough: a player could support himself (and that's not a friendship). So we must also look at the **last previous orders** asking to **MOVE** the unit towards the support's **target** destination. This will link to a 'unit_id' (the one that followed this order) and therefore giving access to the country who made the call.

In [6]:
# first we must look at the supports that happened in this game.
supports = orders.unit_order == "SUPPORT"
orders_w_supports = orders[supports]
orders_w_supports.sample(3)

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num
11452939,37317,30,SUPPORT,Kiel,Munich,Munich,1,None,33
11451916,37317,18,SUPPORT,Rumania,Warsaw,Galicia,0,Support cut by a bud - rum,11
11452602,37317,40,SUPPORT,Ukraine,Moscow,Sevastopol,0,Support cut by a gal - ukr,26


In [7]:
# we want to find the countries of the supported units
# let's take one and see what we can do
support_order = orders_w_supports.iloc[-4]
# support_order = orders_w_supports.head(3).tail(1)
support_order

game_id                             37317
unit_id                                12
unit_order                        SUPPORT
location                          Galicia
target                             Vienna
target_dest                       Bohemia
success                                 0
reason         Support cut by a war - gal
turn_num                               33
Name: 11452914, dtype: object

In [8]:
# Example: there is a support from 'Vienna' to 'Bohemia' 
# we know that in one of the previous orders, someone made a move with destination 'Vienna'
target = support_order.target#.values[0]
turn_number = support_order.turn_num#.values[0]
move_order = orders.query("unit_order == 'MOVE' & target == '{}' & turn_num < {}".format(target, turn_number)).tail(1)
move_order

,game_id,unit_id,unit_order,location,target,target_dest,success,reason,turn_num
11451517,37317,20,MOVE,Galicia,Vienna,None,0,Bounced,3


In [9]:
unit_id = move_order.unit_id.values[0]
move_unit = units.query("unit_id == {}".format(unit_id))
move_unit

,game_id,country,type,start_turn,end_turn,unit_id
1035275,37317,R,A,0,45,20


We see that rusian was the country who had moved it's army there the last time before a support happened. Hence, 'Russia is the supported Country'.

In [11]:
# Let's look at the country who did the support
unit_id = support_order.unit_id.values[0]
support_unit = units.query("unit_id == {}".format(unit_id))
support_unit

AttributeError: 'numpy.int64' object has no attribute 'values'

As it turns out this **is** an act of friendship: Russia was supported by Austria, when it moved from Vienna to Bohemia, by Austrian soldiers who were in Galicia. As we can see on the map, this is perfectly coherent with the geographical position of provinces.

We want to achieve this sort of 'filter' for each of the supports done. Let's see how we can achieve this.

In [ ]:
def is_support_act_of_friendship(support_order):
    # get the last move order to this province
    move_orders = orders.query("unit_order == 'MOVE' & target == '{}' & turn_num < {}"
                        .format(support_order.target, support_order.turn_num))
    if len(move_orders):
        move_order = move_orders.iloc[-1]
        # find the contry that passed this MOVE order
        supported_country = units[["unit_id", "country"]].query("unit_id == {}".format(move_order.unit_id)).country.item()
        # compare it with the country that passed the SUPPORT order
        supporting_country = units[["unit_id", "country"]].query("unit_id == {}".format(support_order.unit_id)).country.item()
        return supported_country != supporting_country
    else:
        return False

In [ ]:
# let's just time this 
s = time.time()
orders_w_supports[["target","turn_num","unit_id"]].apply(is_support_act_of_friendship, axis = 1)
e = time.time()
print("Elapsed:" , e - s)

In [ ]:
orders_w_supports[["target","turn_num","unit_id"]].apply(is_support_act_of_friendship, axis = 1)

Good news : it's working. 

Not as good news: it's rather slow. 

### Finding **acts of hostility**

The code is quite the same, just the logic is a little tweaked. 

We are looking at all **orders** with **unit_order** that is **MOVE** where the **target** of the order is a province, where there is a unit of another player, or in another term: 